# Dataset

In [ ]:
import pandas as pd
df = pd.read_csv('data_with_tag.csv')
df.loc[df['지역'].isnull(), '지역'] = '강북구'
df.head()

place_name_list = df['명칭']
address_num_list = df['지번']
address_name_list = df['도로명']
info_list = df['대본']
category_list = df['유형']

## mysql

In [29]:
import pymysql
import os

mysql_HOST = os.environ.get('mysql_HOST')
mysql_DBNAME = os.environ.get('mysql_DBNAME')
mysql_USER = os.environ.get('mysql_USER')
mysql_PASSWORD = os.environ.get('mysql_PASSWORD')
conn = pymysql.connect(host=mysql_HOST, user=mysql_USER, password=mysql_PASSWORD, db=mysql_DBNAME, charset='utf8')

In [31]:
# 관련 태그 이름을 가지고 있는 장소 정보 가져오기
import pymysql
value='우이'

with conn.cursor() as cur:
    cur.execute(f"""select DISTINCT p.place_id, p.place_name, p.address_name, p.address_num, p.info, p.category, p.picture
from place p
inner join
(select p_h.PLACE_ID, p_h.TAG_ID
FROM place_hashtag p_h
inner join
(select h.tag_id, h.data
from hashtag as h
where h.data like '%{value}%') t1
on p_h.TAG_ID = t1.tag_id) t2
on p.place_id = t2.place_id; """)
    result = cur.fetchall()
    for data in result:
        print(data)
            

(1, '우이동계곡', '강북구 삼양로159길 60-7', '강북구 우이동 산 40-1', "'우이동'은 도봉산과 북한산 사이의 계곡에 위치한 시민의 휴식처입니다. 우이동 입구에서 왼쪽으로는 백운대로 오르는 등산로가 있고, 도선사를 경유하여 인수봉 정상을 오를 수 있습니다. 계곡의 맑은 물을 따라 각종 별미를 자랑하는 음식점들이 줄을 이어 여름철엔 닭죽도 진리라고 해요. 민박도 가능하며, 3월, 4월에는 대학생들의 MT 장소로 선호됩니다.\r\n\r\n20분 정도 오르면 원불교 수도원, 성불사, 용덕사, 선운사, 광명사 등의 사찰을 찾을 수 있습니다. 입구에서의 분위기와는 달리 조용하고 차분한 분위기에서 또 다른 운치를 느낄 수 있어요. 산 정상에는 현재 군부대가 상주하고 있으며, 생태계가 서울에서도 드물게 잘 보존되는 우이령, 일명 바위고개로 이어져 있습니다.\r\n\r\n우이동에서는 우이암으로 이어지는 등산로가 열려 있습니다. 우이암까지 가는 길은 꽤 험하니 주의해 주세요. 우이암 옆을 지나 도봉산 주릉으로 올라가거나, 성황당, 도봉유원지로 하산하시면 됩니다.\r\n", '관광지', '')
(25, '우이동 먹거리마을', '강북구 삼양로181길 45', '강북구 우이동 216-14', '우이동은 도봉산과 북한산 사이의 계곡에 있는 시민들의 휴식처입니다. 우이동 입구에서 왼쪽으로 올라가면 백운대로 오르는 등산길이 있습니다. 도선사를 경유하여 인수봉 정상까지 오를 수도 있습니다. 20분 정도 오르면 원불교 수도원, 성불사, 용덕사, 선운사, 광명사 등 사찰을 방문할 수 있습니다. 입구에서의 분위기는 들뜨지만, 산을 오르면서 차분하고 또 다른 매력을 느낄 수 있습니다. 산 정상에는 군부대가 상주하고 있으며, 이곳은 서울에서도 드물게 잘 보존된 생태계인 우이령, 일명 바위고개로 이어집니다. 우이동에서는 우이암으로 이어지는 등산로도 있습니다. 우이암까지 가는 길은 꽤 험난하지만, 우이암 옆을 지나 도봉산 주릉으로 올라가게 됩니다. 여기서 북쪽으로 뻗은 주릉을 타거나

## Insert

In [ ]:
# Insert Place
with conn.cursor() as cur:
    for place_name, address_num, address_name, info, category in zip(place_name_list, address_num_list, address_name_list, info_list, category_list):
        cur.execute(f"insert into Place (place_name, address_num, address_name, info, category, picture) values ('{place_name}', '{address_num}', '{address_name}', '{info}', '{category}', '');")
        print(place_name)
    conn.commit()

In [ ]:
# Insert Hashtag
tags = []
for i in range(df.shape[0]):
    tags += eval(df['태그only'][i])
    tags = list(set(tags)) 
tags.sort()
tags

with conn.cursor() as cur:
    for tag in tags:
        cur.execute(f"insert into Hashtag (data) values ('{tag}');")
    conn.commit()

# Insert Place_Hashtag
with conn.cursor() as cur:
    place_tag = dict()
    for i in range(df.shape[0]):
        for j in eval(df.loc[i,'태그only']):
            cur.execute(f"insert into Place_Hashtag (PLACE_ID, TAG_ID) values ('{i+1}', '{tags.index(j)+1}');")
                        
    conn.commit()

# postgres

## postgres 데이터 가져오기

In [ ]:
import psycopg2
import os

DB_HOST = os.environ.get('DB_HOST')
DB_DBNAME = os.environ.get('DB_DBNAME')
DB_USER = os.environ.get('DB_USER')
DB_PASSWORD = os.environ.get('DB_PASSWORD')
DB_PORT = os.environ.get('DB_PORT')

conn = psycopg2.connect(host=DB_HOST, dbname=DB_DBNAME, user=DB_USER, password=DB_PASSWORD, port=DB_PORT)
with conn.cursor() as cur:
    cur.execute("select * from place")
    result = cur.fetchall()
    for data in result:
        print(data)

In [33]:
# 관련 태그 이름을 가지고 있는 장소 정보 가져오기
import psycopg2

value='우이'
with conn.cursor() as cur:
    cur.execute(f"""select DISTINCT p.place_id, p.place_name, p.address_name, p.address_num, p.info, p.category, p.picture
                from place p
                inner join
                (select p_h.PLACE_ID, p_h.TAG_ID
                FROM place_hashtag p_h
                inner join
                (select h.tag_id, h.data
                from hashtag as h
                where h.data like '%{value}%') t1
                on p_h.TAG_ID = t1.tag_id) t2
                on p.place_id = t2.place_id; """)
    result = cur.fetchall()
    for data in result:
        print(data)

(1, '우이동계곡', '강북구 삼양로159길 60-7', '강북구 우이동 산 40-1', "'우이동''은 도봉산과 북한산 사이의 계곡에 위치한 시민의 휴식처입니다. 우이동 입구에서 왼쪽으로는 백운대로 오르는 등산로가 있고, 도선사를 경유하여 인수봉 정상을 오를 수 있습니다. 계곡의 맑은 물을 따라 각종 별미를 자랑하는 음식점들이 줄을 이어 여름철엔 닭죽도 진리라고 해요. 민박도 가능하며, 3월, 4월에는 대학생들의 MT 장소로 선호됩니다.\r\n\r\n20분 정도 오르면 원불교 수도원, 성불사, 용덕사, 선운사, 광명사 등의 사찰을 찾을 수 있습니다. 입구에서의 분위기와는 달리 조용하고 차분한 분위기에서 또 다른 운치를 느낄 수 있어요. 산 정상에는 현재 군부대가 상주하고 있으며, 생태계가 서울에서도 드물게 잘 보존되는 우이령, 일명 바위고개로 이어져 있습니다.\r\n\r\n우이동에서는 우이암으로 이어지는 등산로가 열려 있습니다. 우이암까지 가는 길은 꽤 험하니 주의해 주세요. 우이암 옆을 지나 도봉산 주릉으로 올라가거나, 성황당, 도봉유원지로 하산하시면 됩니다.", '관광지', 'https://storage.googleapis.com/largo-storage/%E1%84%8B%E1%85%AE%E1%84%8B%E1%85%B5%E1%84%83%E1%85%A9%E1%86%BC%E1%84%80%E1%85%A8%E1%84%80%E1%85%A9%E1%86%A8.jpg')
(17, '보광사', '강북구 삼양로145길 176', '강북구 우이동 76-19', '안녕하세요! 보광사는 서울시 강북구 우이동 76-19번지 삼각산에 있는 재단법인 선학원 소속사찰입니다. 소나무가 빼곡히 들어서 숲을 이룬 우이동의 솔밭공원을 지나 포장된 길을 따라 걷다보면 삼각산 기슭에 자리한 보광사에 도착합니다. 보광사는 1788년 금강산에서 수도한 원담스님이 신원사를 창건하였고, 이후 한국전쟁으로 인해 소실되었습니다. 하지만 1979년 남산당 정일 스님이 포교의 원을 세우고 이곳에 주석하면서 

In [3]:
with conn.cursor() as cur:
    cur.execute("select * from place_hashtag")
    result = cur.fetchall()
    for data in result:
        print(data)

with conn.cursor() as cur:
    cur.execute("select * from hashtag")
    result = cur.fetchall()
    for data in result:
        print(data)


## postgres의 hashtag, place_hashtag 테이블에 데이터 넣기

In [ ]:
# Insert Hashtag
tags = []
for i in range(df.shape[0]):
    tags += eval(df['태그only'][i])
    tags = list(set(tags)) 
tags.sort()
tags

with conn.cursor() as cur:
    for tag in tags:
        cur.execute(f"insert into Hashtag (data) values ('{tag}');")
    conn.commit()


# Insert Place_Hashtag
with conn.cursor() as cur:
    place_tag = dict()
    for i in range(df.shape[0]):
        for j in eval(df.loc[i,'태그only']):
            cur.execute(f"insert into Place_Hashtag (PLACE_ID, TAG_ID) values ('{i+1}', '{tags.index(j)+1}');")
    conn.commit()